In [190]:
import pandas as pd
import re

from maple.types import Action, ActionType
from maple.classification import parse

import warnings
warnings.filterwarnings('ignore')


In [103]:
legislators = [
"Gomez",
"Cronin",
"Gobi",
"Crighton",
"Tarr",
"Stone", 
"Creem",
"Lesser",
"Chandler",
"Boncore",
"Eldridge",
"Lewis",
"Lovely",
"Comerford",
"Keenan",
"Spilka",
"Brady",
"Montigny",
"Moore",
"Pacheco",
"Rodrigues",
"Rush",
"O'Connor",
"Jehlen",
"Rausch",
"Fattman",
"Chang-Diaz",
"DiDomenico",
"Moran",
"Brownsberger",
"Timilty",
"Consalvo",
"Duffy",
"Giannino",
"Kushmerek",
"Pease",
"Philips",
"Scanlon",
"Xiarhos",
"Oliveira",
"Cabral",
"D'Emilia",
"Madaro",
"Ferrante",
"Michlewitz",
"Peisch",
"Puppolo",
"Silvia",
"Ashe",
"Ayers",
"Jones",
"Murray",
"González",
"Doherty",
"Barber",
"Cronin",
"Fiola",
"LeBoeuf",
"Cahill",
"Donahue",
"Garlick",
"Gregoire",
"Linsky",
"Tucker",
"Balser",
"Holmes",
"Mariano",
"Dooley",
"Garballey",
"Harrington",
"Howitt",
"Peake",
"Pignatelli",
"Ultrino",
"Whipps",
"Farley-Bouvier",
"Golden",
"Nguyen",
"Stanley",
"Walsh",
"Whelan",
"Galvin",
"Straus",
"Turco",
"Fernandes",
"Cassidy",
"Kane",
"Hawkins",
"Arciero",
"Wagner",
"Kelcourse",
"McGonagle",
"McKenna",
"O'Day",
"Parisella",
"Roy",
"Zlotnik",
"Lipper-Garabedian",
"Ferguson",
"Gordon",
"Hogan",
"Campbell",
"Ehrlich",
"Sabadosa",
"Malia",
"Mirra",
"Devers",
"Robinson",
"Cusack",
"Day",
"Decker",
"DuBois",
"Finn",
"Keefe",
"Muratore",
"Blais",
"Higgins",
"Boldyga",
"Donato",
"Durant",
"Haddad",
"Mark",
"Schmid",
"Frost",
"Belsito",
"Hinds",
"Finegold",
"Friedman",
"DiZoglio",
"Kennedy",
"Velis",
"Cyr",
"Barrett",
"Collins",
"Feeney",
"Diggs",
"Oakley",
"Ramos",
"Sullivan",
"Vargas",
"Williams",
"Hendricks",
"Minicucci",
"Sena",
"Berthiaume",
"Robertson",
"Rogers",
"Mom",
"Gouveia",
"Moran",
"Lewis"
]


test_string = """16302,Amended (Comerford) by striking out all after the enacting clause and inserting in place thereof the text of n,,uncategorized,,
17190,Amended (DiDomenico) by striking out all after the enacting clause and inserting in place thereof the text of n,,uncategorized,,
20305,Amended (Lewis-Tarr) by striking out all after the enacting clause and inserting in place thereof the text of n,,uncategorized,,
41667,Amended (Tarr),,uncategorized,,"""

ts2 = "Committee of conference (Finegold-Creem-Fattman) appointed,,uncategorized,,"

hash_reg = re.compile('\(#.*\)')

def normalizeLegislatorNames(action: str):
  matches = [l for l in legislators if l in action]
  
  for m in matches: 
    r = '\(.*' + m + '.*\)'
    action = "(###)".join(re.compile(r).split(action))
  action = "(leg_names)".join(hash_reg.split(action))
  return action

normalizeLegislatorNames(ts2)
  

'Committee of conference (leg_names) appointed,,uncategorized,,'

In [110]:
JointComnittees  = [
        "advanced information technology the internet and cybersecurity",
        "bonding capital expenditures and state assets", "cannabis policy" ,
        "children families and persons with disabilities",
        "community development and small businesses",
        "consumer protection and professional licensure",
        "covid-19 and emergency preparedness and management",
        "economic development and emerging technologies",
        "education",
        "elder affairs",
        "election laws",
        "environment natural resources and agriculture",
        "export development",
        "financial services",
        "health care financing",
        "higher education",
        "housing",
        "the judiciary",
        "labor and workforce development",
        "mental health substance use and recovery",
        "municipalities and regional government",
        "public health",
        "public safety and homeland security",
        "public service",
        "racial equity civil rights and inclusion ",
        "revenue",
        "state administration and regulatory oversight",
        "telecommunications utilities and energy",
        "tourism arts and cultural development",
        "transportation"
]


ts = """3061,Discharged to the committee on Labor and Workforce Development,Labor and Workforce Development,uncategorized,,
199,Discharged to the committee on Mental Health Substance Use and Recovery,Mental Health Substance Use and Recovery,uncategorized,,
17682,Discharged to the committee on Municipalities and Regional Government,Municipalities and Regional Government,uncategorized,,
11091,Discharged to the committee on Public Health,Public Health,uncategorized,,
6369,Discharged to the committee on Public Safety and Homeland Security,Public Safety and Homeland Security,uncategorized,,"""


def extract_committees(action):
  committees = []
  for c in JointComnittees:
    committees.extend(re.compile('(' + c + ')', re.I).findall(action))
  if len(committees) > 0: 
    return committees 


extract_committees(ts)


['Labor and Workforce Development',
 'Labor and Workforce Development',
 'Mental Health Substance Use and Recovery',
 'Mental Health Substance Use and Recovery',
 'Municipalities and Regional Government',
 'Municipalities and Regional Government',
 'Public Health',
 'Public Health',
 'Public Safety and Homeland Security',
 'Public Safety and Homeland Security']

In [206]:
df = pd.read_csv("predictions.csv")

df = df.assign(label=pd.Series(), sublabel=pd.Series())


df['clean_actions'] = df['action'].str.replace(r'[AP]M','xM').str.replace(r'[,.]', '')
df['clean_actions'] = df['clean_actions'].str.replace(r'(?: ?[HS]\d+)+', ' n')
df['clean_actions'] = df['clean_actions'].str.replace(r'\d+ YEAS to \d+ NAYS \(.*$', 'vote')
df['clean_actions'] = df['clean_actions'].str.replace(r'(?:\d+ ?)+ ?(?: ?and \d+)?', 'n ', regex=True)
df['clean_actions'] = df['clean_actions'].str.replace(r'Amendment (?:#n )?\(.*\)', 'Amendment', regex=True)
df['clean_actions'] = df['clean_actions'].str.replace(r'-+ ?see? Roll Call.*$', 'rollcall', regex=True)
df['clean_actions'] = df['clean_actions'].str.replace(r'in the ', 'in ', regex=True)
df['clean_actions'] = df['clean_actions'].str.replace(r'as a ', 'as ', regex=True)
df['clean_actions'] = df['clean_actions'].str.replace(r'of an ', 'of ', regex=True)
df['clean_actions'] = df['clean_actions'].str.replace(r'Reported from the committee on .*', 'Reported from the committee on *', regex=True)
df['clean_actions'] = df['clean_actions'].str.replace(r'The following amendments were considered as one and (\w*)', lambda x: f'The following amendments were considered as one and {x.group(1)}', regex=True)
df['clean_actions'] = df['clean_actions'].apply(lambda s: normalizeLegislatorNames(s))

df['committee'] = df['clean_actions'].apply(lambda s: extract_committees(s))


df.groupby(['clean_actions']).count().sample(30)

,action_id,action,prediction,label,sublabel,committee
clean_actions,,,,,,
Placed in Orders of the day for Thursday April n,1,1,1,0,0,0
Referred to the committee on Children Families and Persons with Disabilities,179,179,179,0,0,179
Postponed to the next session,6,6,6,0,0,0
Reporting date extended to Tuesday May n pending concurrence,104,104,104,0,0,0
Bill reported favorably and referred to the committee on Senate Rules,1,1,1,0,0,0
For message see n Attachment U,1,1,1,0,0,0
Read; and referred to the committee on House Steering Policy and Scheduling,80,80,80,0,0,0
Committee of conference appointed in concurrence - (leg_names),4,4,4,0,0,0
Committee reported ought to be adopted,153,153,153,0,0,0


In [ ]:



drp = df.drop_duplicates(subset='clean_actions')

drp = drp.sort_values(['clean_actions'])

drp.to_csv(path_or_buf='predictions_condensed.csv', columns=['action_id', 'clean_actions','committee', 'prediction','label', 'sublabel'])

In [196]:

cdf = pd.read_csv('./predictions_condensed.csv')



In [200]:
pd.set_option('display.max_colwidth', -1)

ids = {
  16054: "",
  39583: "",
  36: "",   
  1295: "", 
  85: "",   
  1318: "", 
  44231: "",
  28: "",   
  26826: "",
  29916: "",
  16565: "",
  16602: "",
  20445: "",
  20453: "",
  23265: "",
  22348: "",
  17390: "",
  17147: "",
  21584: "",
  23921: "",
  41668: "",
  16303: "",
  30700: "",
  15969: "",
  39550: "",
  4537: "", 
  17305: "",
  19209: "",
  17152: "",
  6027: "", 
  6288: "", 
  1099: "", 
  25376: "",
  1100: "", 
  14669: "",
  15712: "",
  20993: "",
  1862: "", 
  42655: "",
  24819: "",
  40504: "",
  39746: "",
  28239: "",
  14672: "",
  15954: "",
  15955: "",
  20385: "",
  17295: "",
  16075: "",
  16610: "",
}

label = {
  16054: 'published as amended',
  39583: 'concurrence',
  36: ['accompanied', 'new draft'],
  1295: ['accompanied', 'study order'],
  85: ['accompanied', 'by'],
  1318: ['accompanied', 'by'],
  44231: ['accompanied', 'by'],
  28: 'accompanied',
  26826: 'accompanied',
  9: 'accompanied',
  10: 'adopted',
  11: ['adopted', 'vote'],
  12: ['adopted', 'as amended'],
  13: ['adopted', 'concurrence'],
  14: ['adopted', 'concurrence', 'vote'],
  15: ['adopted', 'concurrence', 'vote'],
  16: ['enacted', 'laid before governor'],
  17: ['reference'],
  18: ['reference'],
  19: ['amended', 'as recommended', 'ways and means'],
  20: 'amended',
  21: ['amended', 'text replaced'], 
  22: ['amended', 'text replaced', 'as recommended', 'rules committee'],
  
}

for ln in label: 
  cdf['label'].loc[ln] = label[ln]


display(cdf[cdf.label.isna()][['Unnamed: 0', 'clean_actions', 'prediction', 'label']].head(5)) 
display(cdf.head(30))

# print(cdf['action_id'].head(50))


,Unnamed: 0,clean_actions,prediction,label
23,15968,Amended by striking out all after the enacting clause and inserting in place thereof he text of n,uncategorized,NaN
24,39549,Amended by striking out all after the enacting clause and inserting in place thereof the text of House document numbered n,uncategorized,NaN
25,4536,Amended by striking out all after the enacting clause and inserting in place thereof the text of n,uncategorized,NaN
26,17304,Amended by striking out all after the enacting clause and inserting in place thereof the text of n amended,uncategorized,NaN
27,19208,Amended by striking out all after the enacting clause and inserting in place thereof the text of n as amended,uncategorized,NaN


,Unnamed: 0,action_id,clean_actions,committee,prediction,label,sublabel
0,16053,16054,n published as amended,NaN,uncategorized,published as amended,NaN
1,39582,39583,Accepted in concurrence,NaN,uncategorized,concurrence,NaN
2,35,36,Accompanied a new draft see n,NaN,reference,"[accompanied, new draft]",NaN
3,1294,1295,Accompanied a study order see n,NaN,study_order,"[accompanied, study order]",NaN
4,84,85,Accompanied by n,NaN,reference,"[accompanied, by]",NaN
5,1317,1318,Accompanied by n and n,NaN,reference,"[accompanied, by]",NaN
6,44230,44231,Accompanied by n and n and n,NaN,reference,"[accompanied, by]",NaN
7,27,28,Accompanied n,NaN,reference,accompanied,NaN
8,26825,26826,Accompanied n and n,NaN,reference,accompanied,NaN
9,29915,29916,Accompanying n,NaN,uncategorized,accompanied,NaN


In [198]:
cdf.to_csv(path_or_buf='predictions_condensed.csv', columns=['action_id', 'clean_actions','committee', 'prediction','label', 'sublabel'])
